In [1]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

from concurrent.futures import ThreadPoolExecutor, wait

In [17]:
# Парсим через pandas новости и рейтинги

def hacker_news(pages=10):

    news = pd.DataFrame(columns=['news', 'points'])

    def get_info(page=1):
        url = f'https://news.ycombinator.com/?p={page}'

        responce = requests.get(url)

        # Получаем заголовок
        page = pd.read_html(responce.text)[2]
        page.drop([1], axis=1, inplace=True)
        page.rename(columns={0: 'index', 2: 'news'}, inplace=True)

        page = page[(~page['news'].isna()) & (page['news'] != 'More')]

        # Получаем рейтинг
        points = pd.DataFrame(
            page['news'][page['index'].isna()].reset_index(drop=True))
        points.rename(columns={'news': 'points'}, inplace=True)

        # Проверяем есть ли у новости рейтинг, если его нет то присваиваем ноль
        points['contains_points'] = points['points'].str.contains("points")
        points['points'] = points['points'].str.split(' ')

        def set_points(row):
            if row['contains_points']:
                return row[0][0]
            else:
                return 0

        points['points'] = points.apply(set_points, axis=1)

        # Прикручиваем рейтинг к новости
        news = pd.DataFrame(
            page['news'][~page['index'].isna()].reset_index(drop=True))

        result = pd.concat([news, points['points']], axis=1)

        soup = BeautifulSoup(responce.text, 'html.parser')

        # Начинаем собирать ссылки
        link_list = []

        for i in range(len(soup.find_all(class_='title'))):
            try:
                link = soup.find_all(class_='title')[i].find('a')['href']
                text = soup.find_all(class_='title')[i].text

                if text != 'More':
                    link_list.append([text, link])
                else:
                    pass
            except:
                pass

        links = pd.DataFrame(link_list)
        links.rename(columns={0: 'news', 1: 'link'}, inplace=True)

        # Прикручиваем ссылки
        result = pd.merge(left=result, right=links)
        #result.reset_index(drop=True, inplace=True)

        return result

    for i in range(pages):
        news = pd.concat([news, get_info(i+1)])
    
    news.reset_index(drop=True, inplace=True)
        
    return news

In [18]:
test = hacker_news()

In [19]:
test

,news,points,link
0,Tutanota Is Now Tuta (tuta.com),88,https://tuta.com/blog/tutanota-is-now-tuta
1,You don't need to work on hard problems (2020)...,61,https://www.benkuhn.net/hard/
2,The coolest robot I've ever built (youtube.com),72,https://www.youtube.com/watch?v=bO-DWWFolPw
3,PostgreSQL Encryption: The Available Options (...,76,https://www.hezmatt.org/~mpalmer/blog/2023/11/...
4,Show HN: Patterns – Habit Tracker App (apps.ap...,15,https://apps.apple.com/ua/app/patterns-habit-t...
...,...,...,...
294,Opium production in Afghanistan drops by 95%: ...,48,https://www.unodc.org/documents/crop-monitorin...
295,Epic targets Google app store after losing to ...,56,https://www.washingtonpost.com/technology/2023...
296,How to stop adulterated turmeric from killing ...,267,https://www.economist.com/leaders/2023/11/02/h...
297,Amazon Is a Ripoff (pluralistic.net),30,https://pluralistic.net/2023/11/06/attention-r...


---

In [4]:
url = f'https://news.ycombinator.com/?p={1}'

In [5]:
response = requests.get(url)

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')


In [7]:
link_list = []
for i in range(len(soup.find_all(class_='title'))):
    try:
        link = soup.find_all(class_='title')[i].find('a')['href']
        text = soup.find_all(class_='title')[i].text
        
        if text != 'More':
            link_list.append([text, link])
        else:
            pass
    except:
        pass
    
links = pd.DataFrame(link_list)
links.rename(columns={0:'news', 1:'link'}, inplace=True)
links

,news,link
0,Tutanota Is Now Tuta (tuta.com),https://tuta.com/blog/tutanota-is-now-tuta
1,You don't need to work on hard problems (2020)...,https://www.benkuhn.net/hard/
2,The coolest robot I've ever built (youtube.com),https://www.youtube.com/watch?v=bO-DWWFolPw
3,PostgreSQL Encryption: The Available Options (...,https://www.hezmatt.org/~mpalmer/blog/2023/11/...
4,Show HN: Patterns – Habit Tracker App (apps.ap...,https://apps.apple.com/ua/app/patterns-habit-t...
5,WeWork Goes Bankrupt (bloomberg.com),https://www.bloomberg.com/news/articles/2023-1...
6,Exploring the Macintosh ROM (2019) (macgui.com),https://macgui.com/news/article.php?t=493
7,Bhutan sterilised and vaccinated 100% of their...,https://worldanimalnews.com/the-kingdom-of-bhu...
8,IBM Rebus (ibm.com),https://www.ibm.com/design/language/ibm-logos/...
9,Show HN: Inshellisense – IDE style shell autoc...,https://github.com/microsoft/inshellisense


In [8]:
pd.merge(left=news, right=links)

,news,points,link
0,Tutanota Is Now Tuta (tuta.com),83,https://tuta.com/blog/tutanota-is-now-tuta
1,You don't need to work on hard problems (2020)...,57,https://www.benkuhn.net/hard/
2,The coolest robot I've ever built (youtube.com),69,https://www.youtube.com/watch?v=bO-DWWFolPw
3,PostgreSQL Encryption: The Available Options (...,73,https://www.hezmatt.org/~mpalmer/blog/2023/11/...
4,Show HN: Patterns – Habit Tracker App (apps.ap...,12,https://apps.apple.com/ua/app/patterns-habit-t...
5,WeWork Goes Bankrupt (bloomberg.com),484,https://www.bloomberg.com/news/articles/2023-1...
6,Exploring the Macintosh ROM (2019) (macgui.com),13,https://macgui.com/news/article.php?t=493
7,Bhutan sterilised and vaccinated 100% of their...,107,https://worldanimalnews.com/the-kingdom-of-bhu...
8,IBM Rebus (ibm.com),110,https://www.ibm.com/design/language/ibm-logos/...
9,Show HN: Inshellisense – IDE style shell autoc...,314,https://github.com/microsoft/inshellisense
